In [369]:
import os
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
from dotenv import load_dotenv
load_dotenv()

True

In [370]:
api_key = os.getenv("OPENAI_API_KEY")

In [371]:
%pip install pyowm

Note: you may need to restart the kernel to use updated packages.


In [372]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination

In [373]:
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [374]:
import os
weather_api = "2ab1d162b2c78c5af62d724770cb73dc"

ONLINE WEATHER API TOOL

In [375]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
import requests
import os
def weather_info(city:str) -> str:

    """
    Search the Weather information of a particular city.
    """
    try:
        weather_api = "790e80903edbab1ce9347ff2a9d2924c"
        weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=weather_api)
        weather_data = weather.run(city)
        return weather_data
    except Exception as e:
        print("ERROR OCCURED WHILE SEARCHING FOR CURRENT WEATHER")

CREATING THE WEATHER AGENT

In [376]:
weather_search_agent = AssistantAgent(
    name = "Weather_Information",
    description = "An agent for searching the weather information of a particular city.",
    model_client = model_client,
    tools = [weather_info],
    reflect_on_tool_use=True,
    system_message = """
    You are a Weather searching agent.
    Your only tool is weather_info - use it to find the current weather for a particular location.
    Depending on the weather, you also suggest appropriate clothing to wear.
    You make only one search call at a time.
    """
)

TOOL TO FIND TOP SEARCH ATTRACTION PLACES

In [377]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper

os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")
search_tool_wrapper = GoogleSerperAPIWrapper(type='search')


def tourist_search(city:str, days:int)-> str:
    """
    Search the best tourist spots and top attractions in the city.
    """
    try:
        results = search_tool_wrapper.run(f"Search for the best tourist spots and top attraction places in the {city} for {days} days.")
        return results
    except Exception as e:
        print("SEARCH RESULT FAILED")

CREATING THE AGENT FOR TOURIST SEARCH ATTRACTIONS

In [378]:
tourist_agent = AssistantAgent(
    name = "tourism_information",
    description = "An agent to find the top tourist spots and attractions",
    model_client = model_client,
    tools = [tourist_search],
    system_message = '''
    You are an intelligent tourist spot and top attraction finder.
    You should be able to find the top tourist spots and top attractions using the following tool-'tourist_search'.
    '''
)

DAY WISE ITENARY

In [379]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper

os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")
search_tool_wrapper = GoogleSerperAPIWrapper(type='search')


def daywise_itenary(city:str, days:int)-> str:
    """
    Create a daywise itenary by considering all the essential requirements below.
    - Core Requirements:
        Schedule: Plan each day strictly between 9:00 AM and 6:00 PM.
        Structure: Provide a logical flow for each day, grouping nearby attractions to minimize travel time.
        Logistics: Include a 1-hour lunch break daily and suggest restaurant types (e.g., local cuisine, cafes). Please also provide estimated travel times between locations.

    - Personalization Details:
        Traveler Profile: I am traveling [e.g., solo, as a couple, with my family (kids aged 7 and 10), with a group of friends].
        Pace: I prefer a [Choose one: relaxed, moderate, or packed] pace.
        Budget: My budget is [Choose one: budget-friendly, mid-range, or luxury]. Please suggest activities and dining options that align with this.

    My Interests (be specific):
    Must-See: [List 1-2 top priorities, e.g., Eiffel Tower, Colosseum].
    General Interests: [List several interests, e.g., historical landmarks, art museums, street food, hiking, local markets, photography spots, modern architecture, unique cafes].

    Things to Avoid: [List anything you dislike, e.g., crowded tourist traps, shopping malls, late nights].
    """
    try:
        results = search_tool_wrapper.run(f"Create the daywise itenary for the {city} for {days} days. The daywise itenary should cover all the important places and travel locations.")
        return results
    except Exception as e:
        print("SEARCH RESULT FAILED")

AGENT FOR DAYWISE ITENARY

In [380]:
daywise_itenaryplanner_agent = AssistantAgent(
    name = "daywiseitenary_information",
    description = "An agent to create daywise itenary.",
    model_client = model_client,
    tools = [daywise_itenary],
    system_message = '''
    You are an intelligent daywise itenary planner agent.

    Create a daywise itenary by considering all the essential requirements below.
    - Core Requirements:
        Schedule: Plan each day strictly between 9:00 AM and 6:00 PM.
        Structure: Provide a logical flow for each day, grouping nearby attractions to minimize travel time.
        Logistics: Include a 1-hour lunch break daily and suggest restaurant types (e.g., local cuisine, cafes). Please also provide estimated travel times between locations.

    - Personalization Details:
        Traveler Profile: I am traveling [e.g., solo, as a couple, with my family (kids aged 7 and 10), with a group of friends].
        Pace: I prefer a [Choose one: relaxed, moderate, or packed] pace.
        Budget: My budget is [Choose one: budget-friendly, mid-range, or luxury]. Please suggest activities and dining options that align with this.

    My Interests (be specific):
    Must-See: [List 1-2 top priorities, e.g., Eiffel Tower, Colosseum].
    General Interests: [List several interests, e.g., historical landmarks, art museums, street food, hiking, local markets, photography spots, modern architecture, unique cafes].

    Things to Avoid: [List anything you dislike, e.g., crowded tourist traps, shopping malls, late nights].

    '''
)

TOOL TO FIND THE BEST HOTELS TO STAY FOR USER IN BUDGET

In [381]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper

os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")
hotel_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def hotel_stays(city:str)-> str:
    """
    Search for the best hotels with good ratings in the city within the user budget.
    Return only the cheap and best budget friendly hotels.
    """
    try:
        results = hotel_tool_wrapper.run(f"Please list out 3 best recommended hotels to stay in {city}.")
        return results
    except Exception as e:
        print("EXCEPTION OCCURED IN FINDING THE HOTELS")


CREATING AGENT TO FIND THE BEST HOTELS FOR THE CITY

In [382]:
hotel_agent = AssistantAgent(
    name = "hotel_information",
    description = "An agent to find the best hotels with good ratings in the city within the user budget.",
    model_client = model_client,
    tools = [hotel_stays],
    system_message = '''
    You are an intelligent best hotel finder within budget with good ratings.
    You should be able to find the best hotels in the city using the tool hotel_stays.
    Remember to generate the hotels that are best and cheaper. 
    Suggest atleast 3 hotels as recommendations.
    
    '''
)

TOOL TO FIND THE DAILY BUDGET OF THE TRIP

In [383]:
def calculate_total_cost(hotel_cost: float, activity_cost: float, transport_cost: float) -> float:
    """
    Calculate the total cost of the trip.

    Args:
    hotel_cost (float): The cost of the hotel stay.
    activity_cost (float): The cost of activities and attractions.  
    transport_cost (float): The cost of transportation (e.g., flights, local transport).
    """
    return hotel_cost + activity_cost + transport_cost

In [384]:
cost_agent = AssistantAgent(
    name = "total_cost",
    description = "An agent to total cost of the trip.",
    model_client = model_client,
    tools = [calculate_total_cost],
    system_message = '''
    Your task is to calculate the total cost of the trip based on the hotel, activity, and transport costs.
    You should use the tool calculate_total_cost to compute the total cost.
    Provide a clear breakdown of the costs and the final total.
    Please include the hotel cost, activity cost, and transport cost in a tabular format so that the user can understand the total cost of the trip.
    Before calculating the total cost, make sure to get the hotel cost, activity cost, and transport cost from the respective agents and also make sure to generate the result in a tabular format.
    '''
)

SUMMARY TOOL TO SUMMARIZE THE ENTIRE TRIP INFORMATION

In [385]:
def create_trip_summary(city: str, days: int, total_cost: float, currency: str) -> str:
    """
    Generates a high-level trip summary for the user.

    Args:
        city (str): Destination city for the trip.
        days (int): Number of days planned for the trip.
        total_cost (float): Total estimated cost of the trip in the specified currency.
        currency (str): Currency code, e.g., USD, EUR, BDT.

    Returns:
        str: Prompt instruction for the LLM to summarize the trip.
    """
    return (
        f"Generate a short and friendly trip summary for a {days}-day trip to {city}. "
        f"The estimated total cost is {total_cost:.2f} {currency}. "
        f"Include the number of days, cost estimate, and encourage the traveler. "
        f"End with a line like: 'Have a wonderful trip to {city}!'"
    )

SUMMARY AGENT 

In [386]:
summary_agent = AssistantAgent(
    name = "trip_summary",
    description = "An agent to summarize the entire trip information.",
    model_client = model_client,
    tools = [calculate_total_cost],
    system_message = '''
    Generate a high-level trip summary for the user.
    The summary should include:
    - Destination city  
    - Number of days planned
    - Weather information and clothing suggestions
    - Top tourist attractions and activities
    - Daywise itinerary 
    - Total cost of the trip.
    '''
)

In [387]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat

planning_agent = AssistantAgent(
    name = "Travel_Planner",
    description = "An agent for Travel Planner, this agent should be the first one to engage when a new task is given.",
    model_client = model_client,
    system_message = '''
    You are a Travel_Planner agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        weather_search_agent: Searches for current weather in a particular city.
        tourist_agent: Searches for top attraction places and tourism spots.
        hotel_agent: Searches the best hotels in the city within user budget.
        daywise_itenaryplanner_agent: Plans for daywise itenary.
        cost_agent: Calculates the total cost of the trip.
        summary_agent: Summarizes the entire trip information.
    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE"
    '''
)

In [388]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
combined_termination = text_mention_termination 

In [389]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [390]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, weather_search_agent, tourist_agent,hotel_agent,daywise_itenaryplanner_agent,cost_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [391]:
task = "I want to travel Thailand for 5 days can you help me find the current weather condition, best tourist attraction places in the city and best hotels to stay in the city.Include daywise itenary as well. My cost should be within 100000 INR. Also calculate the total cost of the trip."

In [392]:
from autogen_agentchat.ui import Console
await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
I want to travel Thailand for 5 days can you help me find the current weather condition, best tourist attraction places in the city and best hotels to stay in the city.Include daywise itenary as well. My cost should be within 100000 INR. Also calculate the total cost of the trip.
---------- TextMessage (Travel_Planner) ----------
To assist you in planning your trip to Thailand, I'll break down the tasks and assign them to the appropriate agents:

1. **weather_search_agent**: Search for the current weather conditions in Thailand.
2. **tourist_agent**: Find the top tourist attraction places and tourism spots in Thailand.
3. **hotel_agent**: Search for the best hotels in Thailand within a budget that fits your overall limit of 100000 INR.
4. **daywise_itenaryplanner_agent**: Plan a 5-day itinerary for your trip to Thailand, considering the top attractions and recommended accommodations.
5. **cost_agent**: Calculate the total estimated cost of the t

TaskResult(messages=[TextMessage(id='8698f532-65bf-4bd0-a04a-c91034d476d1', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 12, 28, 55, 957788, tzinfo=datetime.timezone.utc), content='I want to travel Thailand for 5 days can you help me find the current weather condition, best tourist attraction places in the city and best hotels to stay in the city.Include daywise itenary as well. My cost should be within 100000 INR. Also calculate the total cost of the trip.', type='TextMessage'), TextMessage(id='1db82a81-1f15-48fc-b56d-9b6df00a3239', source='Travel_Planner', models_usage=RequestUsage(prompt_tokens=241, completion_tokens=182), metadata={}, created_at=datetime.datetime(2025, 7, 25, 12, 29, 0, 504619, tzinfo=datetime.timezone.utc), content="To assist you in planning your trip to Thailand, I'll break down the tasks and assign them to the appropriate agents:\n\n1. **weather_search_agent**: Search for the current weather conditions in Thailand.\n2.